In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

dir = '/content/drive/MyDrive/tern_project/Eyal/TrackingTerns'
os.chdir(dir)

### Read parameters from config file

In [3]:
import configparser

# Initialize the ConfigParser object
config = configparser.ConfigParser()
# Read the INI file
config.read('track_breeding_terns.ini')
# A date to track breeding terns on
date = config.get('General', 'date').split(',')
# Path to the input single-scan tracking results directory
one_scan_result_dir = config.get('General', 'one_scan_result_dir')
# Path to the output directory for multi-scan tracking results
mult_scans_result_dir = config.get('General', 'mult_scans_result_dir')
# Labels distribution of flags path
images_dir = config.get('General', 'images_dir')

In [4]:
images_dir

'./../ConvertVideoToImage/ImagesDir'

In [5]:
def isFileFromDates(path, dates):
    return any(date in path for date in dates)

def get_dir_path_with_parent(directory_path):
  # Get the parent directory and the base name of the directory
  parent_dir, dir_name = os.path.split(os.path.normpath(directory_path))

  # Get the parent directory name and concatenate it with the directory name
  return os.path.join("/", os.path.basename(parent_dir), dir_name)[1:]

#### List all tour directories from Yolo result directory

In [6]:
import glob

# Get all directories two levels
two_level_dirs = glob.glob(os.path.join(one_scan_result_dir, '*', '*'))

# Filter out only directories
two_level_dirs = [get_dir_path_with_parent(path) for path in two_level_dirs \
                  if os.path.isdir(path) and isFileFromDates(path, date)]

print('My tour list looks like this:')
two_level_dirs[:]

My tour list looks like this:


['atlitcam181.stream_2024_05_30_10_00_00/tour0',
 'atlitcam181.stream_2024_05_30_10_00_00/tour1',
 'atlitcam181.stream_2024_05_30_15_00_00/tour0',
 'atlitcam181.stream_2024_05_30_15_00_00/tour1',
 'atlitcam191.stream_2024_05_30_10_00_00/tour0',
 'atlitcam191.stream_2024_05_30_10_00_00/tour1',
 'atlitcam191.stream_2024_05_30_15_00_02/tour0',
 'atlitcam191.stream_2024_05_30_15_00_02/tour1']

#### Amaglate tours which are from the same date

In [7]:
# This function extract the camera number (181/191) from the tour string sub path
def get_camera_number(tour):
    return tour.split('.')[0].split('cam')[-1]

# This function Extracts the date from the tour string.
def get_tour_date(tour):
    return tour.split('_', 1)[1][:10]

In [8]:
# Create a defaultdict to store tours by date
tours_grouped_dict = dict()

# Iterate through the tour list and group tours by date
for tour in two_level_dirs:
    key = (get_camera_number(tour), get_tour_date(tour))
    tours_grouped_dict.setdefault(key, []).append(tour)

print('My tours collected by date and camera:')
print(tours_grouped_dict)

My tours collected by date and camera:
{('181', '2024_05_30'): ['atlitcam181.stream_2024_05_30_10_00_00/tour0', 'atlitcam181.stream_2024_05_30_10_00_00/tour1', 'atlitcam181.stream_2024_05_30_15_00_00/tour0', 'atlitcam181.stream_2024_05_30_15_00_00/tour1'], ('191', '2024_05_30'): ['atlitcam191.stream_2024_05_30_10_00_00/tour0', 'atlitcam191.stream_2024_05_30_10_00_00/tour1', 'atlitcam191.stream_2024_05_30_15_00_02/tour0', 'atlitcam191.stream_2024_05_30_15_00_02/tour1']}


In [9]:
tours_grouped_dict.keys()

dict_keys([('181', '2024_05_30'), ('191', '2024_05_30')])

In [10]:
import subprocess

for (cam_name, date), tours_list in tours_grouped_dict.items():
    print(f'Tracking terns on camera {cam_name} on date {date}')
    command1 = ' '.join(["python", "./track_breeding_terns_runner.py"] + tours_list)
    result = subprocess.run(command1, shell=True, capture_output=True, text=True)
    # Check if the command ran successfully
    if result.returncode == 0:
        print(f'Successfully tracking birds...')
    else:
        print(result.stdout)
        print(f'Error occurred while tracking...')
        print(f'STDERR: {result.stderr}')

Tracking terns on camera 181 on date 2024_05_30
Directory './../TrackingTerns/TrackerResults/MultipleScansTracks/2024_05_30_181/flag92' created successfully.
./../ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_05_30_10_00_00/tour0
./../TrackingTerns/TrackerResults/MultipleScansTracks/2024_05_30_181/flag92
flag92_0_atlitcam181.stream_2024_05_30_10_00_00.jpg
atlitcam181.stream_2024_05_30_10_00_00_tour0.jpg
Image 'flag92_0_atlitcam181.stream_2024_05_30_10_00_00.jpg' copied successfully from ./../ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_05_30_10_00_00/tour0 to ./../TrackingTerns/TrackerResults/MultipleScansTracks/2024_05_30_181/flag92.
./../ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_05_30_10_00_00/tour1
./../TrackingTerns/TrackerResults/MultipleScansTracks/2024_05_30_181/flag92
flag92_0_atlitcam181.stream_2024_05_30_10_00_00.jpg
atlitcam181.stream_2024_05_30_10_00_00_tour1.jpg
Image 'flag92_0_atlitcam181.stream_2024_05_30_10_00_00.jpg' copied successfully fro